In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
%matplotlib inline 

/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df1 = pd.read_csv('1year.csv')
df2 = pd.read_csv('2year.csv')
df3 = pd.read_csv('3year.csv')
df4 = pd.read_csv('4year.csv')
df5 = pd.read_csv('5year.csv')

/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,5,6,8,9,10,13,17,21,23,24,28,34,35,37,47,50,54,56,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/fujiwara/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,5,6,7,9,10,13,14,15,16,17,21,23,24,25,28,33,34,35,37,47,49,50,56,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


必要な関数の定義

In [10]:
def dropString(d):
    for _d in d:
        try:
            yield float(_d)
        except ValueError:
            yield np.nan

def getPricePerUnit(TradePrice, Area):
    for t, a in zip(TradePrice, Area):
        try:
            yield float(t) / float(a)
        except ValueError:
            yield np.nan

def getAge(BuildingYear, Period):
    for b, p in zip(BuildingYear, Period):
        try:
            yield int(str(p)[-4:]) - int(b)
        except ValueError:
            yield np.nan

文字列のデータを欠損値に変換

In [11]:
for t in df1.columns:
    df1[t] = list(dropString(df1[t]))
    df2[t] = list(dropString(df2[t]))
    df3[t] = list(dropString(df3[t]))
    df4[t] = list(dropString(df4[t]))
    df5[t] = list(dropString(df5[t]))

欠損値を平均で補間

In [12]:
df1 = df1.fillna(df1.mean())
df2 = df1.fillna(df2.mean())
df3 = df1.fillna(df3.mean())
df4 = df1.fillna(df4.mean())
df5 = df1.fillna(df5.mean())

標準化

In [19]:
stdc = StandardScaler()
_df1 = df1.drop('class', axis=1)
_df2 = df2.drop('class', axis=1)
_df3 = df3.drop('class', axis=1)
_df4 = df4.drop('class', axis=1)
_df5 = df5.drop('class', axis=1)

data1 = pd.DataFrame(stdc.fit_transform(_df1), columns=_df1.columns)
data2 = pd.DataFrame(stdc.fit_transform(_df2), columns=_df2.columns)
data3 = pd.DataFrame(stdc.fit_transform(_df3), columns=_df3.columns)
data4 = pd.DataFrame(stdc.fit_transform(_df4), columns=_df4.columns)
data5 = pd.DataFrame(stdc.fit_transform(_df5), columns=_df5.columns)

data1.describe().to_csv('des1.csv')
data2.describe().to_csv('des2.csv')
data3.describe().to_csv('des3.csv')
data4.describe().to_csv('des4.csv')
data5.describe().to_csv('des5.csv')

data1 = pd.concat([data1, df1['class']], axis=1)
data2 = pd.concat([data2, df2['class']], axis=1)
data3 = pd.concat([data3, df3['class']], axis=1)
data4 = pd.concat([data4, df4['class']], axis=1)
data5 = pd.concat([data5, df5['class']], axis=1)

テスト用データと訓練用データに分割

In [21]:
X1, Y1 = data1.drop('class', axis=1).values, data1['class'].values
X2, Y2 = data2.drop('class', axis=1).values, data2['class'].values
X3, Y3 = data3.drop('class', axis=1).values, data3['class'].values
X4, Y4 = data4.drop('class', axis=1).values, data4['class'].values
X5, Y5 = data5.drop('class', axis=1).values, data5['class'].values

X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1, Y1, test_size=0.3)
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.3)
X3_train, X3_test, Y3_train, Y3_test = train_test_split(X3, Y3, test_size=0.3)
X4_train, X4_test, Y4_train, Y4_test = train_test_split(X4, Y4, test_size=0.3)
X5_train, X5_test, Y5_train, Y5_test = train_test_split(X5, Y5, test_size=0.3)

DataFrameへ変換

In [22]:
data1_train = pd.DataFrame(X1_train, columns= data1.columns.drop('class'))
data2_train = pd.DataFrame(X2_train, columns= data2.columns.drop('class'))
data3_train = pd.DataFrame(X3_train, columns= data3.columns.drop('class'))
data4_train = pd.DataFrame(X4_train, columns= data4.columns.drop('class'))
data5_train = pd.DataFrame(X5_train, columns= data5.columns.drop('class'))

data1_train['class'] = Y1_train
data2_train['class'] = Y2_train
data3_train['class'] = Y3_train
data4_train['class'] = Y4_train
data5_train['class'] = Y5_train

data1_test = pd.DataFrame(X1_test, columns= data1.columns.drop('class'))
data2_test = pd.DataFrame(X2_test, columns= data2.columns.drop('class'))
data3_test = pd.DataFrame(X3_test, columns= data3.columns.drop('class'))
data4_test = pd.DataFrame(X4_test, columns= data4.columns.drop('class'))
data5_test = pd.DataFrame(X5_test, columns= data5.columns.drop('class'))

data1_test['class'] = Y1_test
data2_test['class'] = Y2_test
data3_test['class'] = Y3_test
data4_test['class'] = Y4_test
data5_test['class'] = Y5_test

HDF形式で保存

In [24]:
data1_train.to_hdf('data.h5', key='data1_train')
data2_train.to_hdf('data.h5', key='data2_train')
data3_train.to_hdf('data.h5', key='data3_train')
data4_train.to_hdf('data.h5', key='data4_train')
data5_train.to_hdf('data.h5', key='data5_train')

data1_test.to_hdf('data.h5', key='data1_test')
data2_test.to_hdf('data.h5', key='data2_test')
data3_test.to_hdf('data.h5', key='data3_test')
data4_test.to_hdf('data.h5', key='data4_test')
data5_test.to_hdf('data.h5', key='data5_test')